In [1]:
%run "10_Data into model format.ipynb"

function: load_DProcessed_df(file), amex_metric(y_true, y_pred), file_get(dataset,version,dataset_list,num_list_inModel,ignore_list)
list:all_cols,cat_list,num_list , varible: Huge_data_path


In [2]:

params = {
'objective':'binary',
#'boosting': 'dart',
'random_state': 19,
"num_leaves": 50,#100
'learning_rate': 0.03,#0.03
'colsample_bytree': 0.19,
'n_estimators': 6000,#1200
'max_bins': 60,
'importance_type':'gain',
'min_child_samples':500,
'early_stopping_round':100,
'verbose':-1,
#'reg_lambda':50,
'n_jobs':7,
'device':'gpu',
             }

applied_items = dict()
applied_r = dict()
word_dict = {
    'baseset_withdroped':['lagmean'],#['first','last','mean','std','min','max'],#'first','last','mean','std'
    'single_lagset':['_lag1','_lag3','_lag6','_lagf'],
    'avgstatset':['_f3','_f6','_f-3','_f-6'],
    'slopeset':['_slope'],
    'PCAset':['PCA'],
    'NAset':['_na'],
    'cat_WoE_baseset':['WoE'],
    #'cat__WoEonlylast_baseset':['WoE'],
    'Woe_slopeset':['slope'],
    'cat_stat':['_'],
    'refeature_items':['_'],
    'acfpacf':['acf','pacf'],
    'LmmMean':['LmmMean'],

}

dataset = 'train'
#for dataset_i in dataset_list:
for dataset_i in ['LmmMean']:
    for word in word_dict[dataset_i]:
        if dataset_i =='cat_stat':
            data_df = file_get_word(dataset,11,[],[word],[dataset_i])
        else:
            data_df = file_get_word(dataset,11,[dataset_i],[word],)
        x_train, x_valid, y_train, y_valid = train_test_split(data_df.drop('target',axis=1),data_df['target'], #
                                                            test_size=0.2,random_state=11)
        #x_valid = x_valid.astype('float16')

        model = lgbm.LGBMClassifier(**params)
        model.fit(x_train,y_train,eval_set=[(x_valid, y_valid)],)
        r = permutation_importance(model, x_valid, y_valid,
                                n_repeats=5,n_jobs = 7,#5
                                random_state=0)

        applied_r[dataset_i+'_'+word] = r.importances_mean
        applied_items[dataset_i+'_'+word] = x_valid.columns
applied_items

[1]	valid_0's binary_logloss: 0.562225
[2]	valid_0's binary_logloss: 0.549615
[3]	valid_0's binary_logloss: 0.537749
[4]	valid_0's binary_logloss: 0.526142
[5]	valid_0's binary_logloss: 0.515551
[6]	valid_0's binary_logloss: 0.505258
[7]	valid_0's binary_logloss: 0.495719
[8]	valid_0's binary_logloss: 0.486743
[9]	valid_0's binary_logloss: 0.477902
[10]	valid_0's binary_logloss: 0.469123
[11]	valid_0's binary_logloss: 0.461633
[12]	valid_0's binary_logloss: 0.453766
[13]	valid_0's binary_logloss: 0.446341
[14]	valid_0's binary_logloss: 0.439749
[15]	valid_0's binary_logloss: 0.432771
[16]	valid_0's binary_logloss: 0.426789
[17]	valid_0's binary_logloss: 0.420534
[18]	valid_0's binary_logloss: 0.415361
[19]	valid_0's binary_logloss: 0.410409
[20]	valid_0's binary_logloss: 0.405252
[21]	valid_0's binary_logloss: 0.400269
[22]	valid_0's binary_logloss: 0.395443
[23]	valid_0's binary_logloss: 0.390741
[24]	valid_0's binary_logloss: 0.386172
[25]	valid_0's binary_logloss: 0.381549
[26]	vali

{'LmmMean_LmmMean': Index(['period_count', 'B_1_LmmMean', 'B_10_LmmMean', 'B_11_LmmMean',
        'B_12_LmmMean', 'B_13_LmmMean', 'B_14_LmmMean', 'B_15_LmmMean',
        'B_16_LmmMean', 'B_18_LmmMean',
        ...
        'D_61_cen_LmmMean', 'D_73_std_LmmMean', 'D_76_std_LmmMean',
        'R_7_Q50std_LmmMean', 'R_9_std_LmmMean', 'S_11_Q50std_LmmMean',
        'S_5_std_LmmMean', 'S_7_cen_LmmMean', 'S_8_std_LmmMean',
        'S_9_std_LmmMean'],
       dtype='object', length=187)}

In [4]:

collect_list = pd.DataFrame()
for dataset in  ['baseset_withdroped']:#dataset_list:
    for word in word_dict[dataset]:
        key = dataset+'_'+word
        sorted_order = pd.DataFrame(applied_r[key],index=applied_items[key],columns=['r']).sort_values('r',ascending=False).drop('period_count')
        sorted_order['dataset_word'] = key
        collect_list = pd.concat([collect_list,sorted_order.head(int(len(sorted_order)*0.2))])
collect_list.to_csv('Data/p_important_listII.csv')
collect_list

,r,dataset_word
D_42_lagmean,0.002724,baseset_withdroped_lagmean
D_44_lagmean,0.002449,baseset_withdroped_lagmean
B_17_lagmean,0.002218,baseset_withdroped_lagmean
B_4_lagmean,0.002205,baseset_withdroped_lagmean
D_39_lagmean,0.001872,baseset_withdroped_lagmean
B_3_lagmean,0.001741,baseset_withdroped_lagmean
D_50_lagmean,0.001604,baseset_withdroped_lagmean
R_1_lagmean,0.001416,baseset_withdroped_lagmean
B_9_lagmean,0.001192,baseset_withdroped_lagmean
D_127_lagmean,0.001124,baseset_withdroped_lagmean


In [5]:

collect_list = pd.DataFrame()
for dataset in  ['LmmMean']:#dataset_list:
    for word in word_dict[dataset]:
        key = dataset+'_'+word
        sorted_order = pd.DataFrame(applied_r[key],index=applied_items[key],columns=['r']).sort_values('r',ascending=False)
        sorted_order['dataset_word'] = key
        collect_list = pd.concat([collect_list,sorted_order.head(int(len(sorted_order)))])
collect_list.to_csv('Data/p_important_fulllistII.csv')
collect_list

,r,dataset_word
P_2_LmmMean,0.007248,LmmMean_LmmMean
B_2_LmmMean,0.004630,LmmMean_LmmMean
B_4_LmmMean,0.003083,LmmMean_LmmMean
D_46_Q50std_LmmMean,0.002449,LmmMean_LmmMean
B_5_LmmMean,0.002144,LmmMean_LmmMean
...,...,...
D_132_LmmMean,-0.000187,LmmMean_LmmMean
B_10_LmmMean,-0.000209,LmmMean_LmmMean
D_106_LmmMean,-0.000209,LmmMean_LmmMean
B_36_LmmMean,-0.000218,LmmMean_LmmMean


In [7]:
collect_list[collect_list['r']>0]

,r,dataset_word
P_2_LmmMean,7.247529e-03,LmmMean_LmmMean
B_2_LmmMean,4.630487e-03,LmmMean_LmmMean
B_4_LmmMean,3.083360e-03,LmmMean_LmmMean
D_46_Q50std_LmmMean,2.449255e-03,LmmMean_LmmMean
B_5_LmmMean,2.144188e-03,LmmMean_LmmMean
...,...,...
S_25_LmmMean,2.179053e-06,LmmMean_LmmMean
D_76_std_LmmMean,2.179053e-06,LmmMean_LmmMean
S_5_std_LmmMean,2.179053e-06,LmmMean_LmmMean
D_86_LmmMean,2.179053e-06,LmmMean_LmmMean


In [13]:
pd.read_csv('Data/p_important_list.csv',index_col=0).set_index('item').drop('Unnamed: 0',axis=1)

,r,dataset_word
item,,
P_2_first,0.011412,baseset_withdroped_first
D_42_first,0.006402,baseset_withdroped_first
D_53_first,0.005411,baseset_withdroped_first
B_17_first,0.004994,baseset_withdroped_first
D_46_first,0.004166,baseset_withdroped_first
...,...,...
B_38_Woeslope,0.021477,Woe_slopeset_slope
B_38_last,0.073016,cat_stat__
B_30_last,0.011015,cat_stat__


In [6]:
p_important_fulllist = pd.concat([pd.read_csv('Data/p_important_fulllist.csv',index_col=0),pd.read_csv('Data/p_important_fulllistII.csv',index_col=0)])
p_important_fulllist.to_csv('Data/p_important_fulllist.csv')
p_important_fulllist

,r,dataset_word
P_2_first,0.011412,baseset_withdroped_first
D_42_first,0.006402,baseset_withdroped_first
D_53_first,0.005411,baseset_withdroped_first
B_17_first,0.004994,baseset_withdroped_first
D_46_first,0.004166,baseset_withdroped_first
...,...,...
D_132_LmmMean,-0.000187,LmmMean_LmmMean
B_10_LmmMean,-0.000209,LmmMean_LmmMean
D_106_LmmMean,-0.000209,LmmMean_LmmMean
B_36_LmmMean,-0.000218,LmmMean_LmmMean


In [5]:
#p_important_listII,p_important_fulllistII

#p_important_list = pd.concat([pd.read_csv('Data/p_important_list.csv',index_col=0).set_index('item').drop('Unnamed: 0',axis=1),pd.read_csv('Data/p_important_listII.csv',index_col=0)])
p_important_fulllist = pd.concat([pd.read_csv('Data/p_important_fulllist.csv',index_col=0).set_index('item').drop('Unnamed: 0',axis=1),pd.read_csv('Data/p_important_fulllistII.csv',index_col=0)])
#p_important_list.to_csv('Data/p_important_list.csv')
p_important_fulllist.to_csv('Data/p_important_fulllist.csv')
p_important_fulllist


KeyError: "None of ['item'] are in the columns"

In [3]:
p_important_fulllist = pd.read_csv('Data/p_important_fulllist.csv',index_col=0)
p_important_fulllist

,r,dataset_word
P_2_first,0.011412,baseset_withdroped_first
D_42_first,0.006402,baseset_withdroped_first
D_53_first,0.005411,baseset_withdroped_first
B_17_first,0.004994,baseset_withdroped_first
D_46_first,0.004166,baseset_withdroped_first
...,...,...
D_46_Q50std_f6avg,-0.000037,refeature_items__
B_17_Q50std_first,-0.000059,refeature_items__
S_11_Q50std_lag1_prop,-0.000078,refeature_items__
D_59_Q50std_f-6std,-0.000096,refeature_items__


In [13]:
p_important_fulllist = pd.concat([p_important_fulllist,pd.DataFrame({'r':np.repeat(0.001,len(acf_set)),'dataset_word':np.repeat('acfpacf_acf',len(acf_set))},index=[col + '_acf' for col in acf_set])])
p_important_fulllist = pd.concat([p_important_fulllist,pd.DataFrame({'r':np.repeat(0.001,len(pacf_set)),'dataset_word':np.repeat('acfpacf_pacf',len(pacf_set))},index=[col + '_pacf' for col in pacf_set])])
p_important_fulllist.to_csv('Data/p_important_fulllist.csv')


,r
dataset_word,
NAset__na,63
PCAset_PCA,13
Woe_slopeset_slope,12
acfpacf_acf,54
acfpacf_pacf,42
avgstatset__f-3,499
avgstatset__f-6,499
avgstatset__f3,499
avgstatset__f6,499


In [19]:
collect_list = pd.read_csv('Data/p_important_fulllist.csv',index_col=0)
file = 'p_feature_important'

#2726 123 1515
# array(['baseset_withdroped_first', 'baseset_withdroped_last',
#        'baseset_withdroped_mean', 'baseset_withdroped_std',
#        'single_lagset__lag1', 'single_lagset__lag3',
#        'single_lagset__lag6', 'single_lagset__lagf', 'avgstatset__f3',
#        'avgstatset__f6', 'avgstatset__f-3', 'avgstatset__f-6',
#        'slopeset__slope', 'PCAset_PCA', 'NAset__na',
#        'cat_WoE_baseset_WoE', 'cat__WoEonlylast_baseset_WoE'],
for i in collect_list['dataset_word'].unique():
    logic = collect_list['dataset_word'] == i
    review_df = collect_list[logic & (collect_list['r']>0)]
    fig , ax = plt.subplots(1,1,figsize=(30, 15))
    ax.set_xticklabels(review_df.index, rotation=90)
    plt.bar(review_df.index,review_df['r'])
    fig.savefig('../huge data source/AE/figure/'+str(file)+'/' + i + '.png',facecolor='white', edgecolor='none')
    print(str(i),end="\r", flush=True)
    plt.close(fig)
    gc.collect()


In [69]:
collect_list['dataset_word'].unique()

array(['baseset_withdroped_first', 'baseset_withdroped_last',
       'baseset_withdroped_mean', 'baseset_withdroped_std',
       'single_lagset__lag1', 'single_lagset__lag3',
       'single_lagset__lag6', 'single_lagset__lagf', 'avgstatset__f3',
       'avgstatset__f6', 'avgstatset__f-3', 'avgstatset__f-6',
       'slopeset__slope', 'PCAset_PCA', 'NAset__na',
       'cat_WoE_baseset_WoE', 'cat__WoEonlylast_baseset_WoE',
       'baseset_withdroped_min', 'baseset_withdroped_max',
       'Woe_slopeset_slope', 'cat_stat__'], dtype=object)

In [23]:
n=10
i = 21
needed_item = 'D_70'
review_coll = collect_list[(collect_list['dataset_word'] == collect_list['dataset_word'].unique()[i])]
print(review_coll.loc[[col for col in review_coll.index if (col.startswith('D_59'))]])
print(review_coll.loc[[col for col in review_coll.index if (col.startswith(needed_item))]])
review_coll[n:n+50]

                            r                dataset_word
D_59_Q50cen_lagmean  0.000734  baseset_withdroped_lagmean
D_59_lagmean         0.000139  baseset_withdroped_lagmean
                     r                dataset_word
D_70_lagmean  0.000087  baseset_withdroped_lagmean


,r,dataset_word
D_53_lagmean,0.000996,baseset_withdroped_lagmean
S_26_lagmean,0.000954,baseset_withdroped_lagmean
B_6_lagmean,0.000944,baseset_withdroped_lagmean
B_16_lagmean,0.000876,baseset_withdroped_lagmean
D_65_lagmean,0.000822,baseset_withdroped_lagmean
B_10_lagmean,0.000789,baseset_withdroped_lagmean
B_5_lagmean,0.000778,baseset_withdroped_lagmean
D_59_Q50cen_lagmean,0.000734,baseset_withdroped_lagmean
S_23_lagmean,0.000708,baseset_withdroped_lagmean
P_2_lagmean,0.000660,baseset_withdroped_lagmean


In [95]:
pd.read_csv('Data/important_scoresplit.csv')

,item,high,mid
0,baseset_withdroped_first,0.00100,0.000100
1,baseset_withdroped_last,0.00050,0.000100
2,baseset_withdroped_mean,0.00070,0.000450
3,baseset_withdroped_std,0.00045,0.000120
4,single_lagset__lag1,0.00045,0.000181
5,single_lagset__lag3,0.00040,0.000100
6,single_lagset__lag6,0.00017,0.000070
7,single_lagset__lagf,0.00023,0.000190
8,avgstatset__f3,0.00029,0.000100
9,avgstatset__f6,0.00070,0.000400
